<a href="https://colab.research.google.com/github/TEAM-N0VA/strat-final/blob/main/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =============================================================================
# 1. 라이브러리 설치 (최초 1회 실행)
# =============================================================================
!pip install flask pyngrok scikit-learn pandas numpy

# =============================================================================
# 2. 서버 및 분석 로직 전체 코드
# =============================================================================
from flask import Flask, request, jsonify
from pyngrok import ngrok
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import io
from google.colab import files

# ---------------------------------------------------------
# Ngrok 토큰 설정
# ---------------------------------------------------------
ngrok.set_auth_token("토큰 입력")

app = Flask(__name__)
port = 5000

# Ngrok 터널 열기
public_url = ngrok.connect(port).public_url
print(f"🚀 [서버 시작] 안드로이드 Retrofit URL에 입력하세요: {public_url}/")

# ---------------------------------------------------------
# 📂 [단계 1] 구독 데이터베이스 CSV 파일 업로드 및 로드
# ---------------------------------------------------------
print("\n📂 준비하신 구독 데이터베이스 CSV 파일을 업로드해주세요!")
print("   (필수 컬럼: service_name, cost, benefit_summary)")
uploaded = files.upload()

REC_LIST = [] # 추천 리스트 저장소

if uploaded:
    filename = next(iter(uploaded))
    try:
        # 한글 인코딩 자동 처리 (utf-8-sig 또는 cp949)
        try:
            sub_db = pd.read_csv(io.BytesIO(uploaded[filename]), encoding='utf-8-sig')
        except:
            sub_db = pd.read_csv(io.BytesIO(uploaded[filename]), encoding='cp949')

        # 데이터프레임을 딕셔너리 리스트로 변환
        # 예: [{'service_name': '요기패스', 'cost': 9900, 'benefit_summary': '배달비 무료'}, ...]
        REC_LIST = sub_db.to_dict('records')
        print(f"✅ CSV 로드 완료! 총 {len(REC_LIST)}개의 추천 서비스가 있습니다.")
        print(sub_db.head()) # 데이터 확인용

    except Exception as e:
        print(f"❌ CSV 파일 읽기 실패: {e}")
        print("기본 하드코딩 데이터베이스를 사용하지 않고 비워둡니다.")
else:
    print("⚠️ 파일이 업로드되지 않았습니다. 추천 기능이 제한됩니다.")


# ---------------------------------------------------------
# 핵심 분석 로직
# ---------------------------------------------------------
def analyze_comprehensive(df):

    # 1. 데이터 전처리 (서비스별 집계)
    summary = df.groupby('serviceName').agg({
        'cost': 'sum',
        'timeMinutes': 'sum',
        'payment_count': 'sum',
        'category': 'first'
    }).reset_index()

    # 2. Feature Engineering (효율성 지수)
    efficiency_scores = []
    for i, row in summary.iterrows():
        cat = row['category']
        cost = row['cost']
        count = row['payment_count']
        time = row['timeMinutes']

        if cat in ['OTT', 'MUSIC', 'SUB_TIME']:
            score = cost / (time + 1)
        else:
            score = 10000 / (count + 0.1)
        efficiency_scores.append(score)

    summary['efficiency_index'] = efficiency_scores

    # 3. AI 모델 적용 (데이터 적으면 규칙 기반)
    if len(summary) < 3:
        summary['cluster_label'] = 0
        summary['anomaly'] = 1
    else:
        # Scaling
        scaler = StandardScaler()
        features = summary[['cost', 'timeMinutes', 'payment_count', 'efficiency_index']]
        X_scaled = scaler.fit_transform(features)

        # K-Means (최적 K 찾기)
        best_k = 2
        best_score = -1
        max_k = min(len(summary), 5)

        if max_k > 2:
            for k in range(2, max_k):
                km = KMeans(n_clusters=k, random_state=42, n_init=10)
                labels = km.fit_predict(X_scaled)
                try:
                    score = silhouette_score(X_scaled, labels)
                    if score > best_score:
                        best_score = score
                        best_k = k
                except:
                    pass

        kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=10)
        summary['cluster_label'] = kmeans.fit_predict(X_scaled)

        # Isolation Forest (이상치 탐지)
        iso_forest = IsolationForest(contamination=0.2, random_state=42)
        summary['anomaly'] = iso_forest.fit_predict(X_scaled)

    # 4. 리포트 생성
    report_list = []
    category_spend = {}

    for i, row in summary.iterrows():
        svc = row['serviceName']
        cat = row['category']
        cost = row['cost']
        count = row['payment_count']
        time = row['timeMinutes']
        is_anomaly = row.get('anomaly', 1)

        category_spend[cat] = category_spend.get(cat, 0) + cost

        status = "적정 사용"
        reason = "평범한 사용 패턴입니다."

        if is_anomaly == -1:
            if cost > 10000 and (count <= 1 and time < 60):
                status = "🔴 낭비 경고 (AI 감지)"
                reason = "비용 대비 사용량이 현저히 낮아 주의가 필요합니다."
            elif count >= 5 or time > 1000:
                status = "💎 압도적 효율 (AI 감지)"
                reason = "남들보다 훨씬 알뜰하게 잘 사용하고 계시네요!"

        elif cat in ['FOOD', 'SHOPPING', 'SUB_AMOUNT']:
            if count >= 5:
                status = "🟢 뽕뽑는 중"
                reason = f"이번 달 {int(count)}회 이용! 본전 뽑았습니다."
            elif cost > 20000 and count <= 1:
                status = "🟡 주의 필요"
                reason = "이용 횟수에 비해 결제 금액이 큽니다."
        else:
            if time < 60 and cost > 5000:
                status = "🟡 주의 필요"
                reason = "사용 시간이 다소 부족해 보입니다."

        report_list.append({
            "service": svc,
            "status": status,
            "reason": reason,
            "detail": f"{int(cost)}원 | {int(count)}회 | {int(time)}분"
        })

    # 5. 페르소나 선정
    if not category_spend:
        persona = "데이터 없음"
    else:
        top_category = max(category_spend, key=category_spend.get)
        if "FOOD" in top_category: persona = "🍗 배달비로 집 한 채 날린 미식가"
        elif "OTT" in top_category: persona = "🎥 넷플릭스가 낳은 괴물"
        elif "SHOPPING" in top_category: persona = "📦 택배 기사님과 절친"
        else: persona = "🎧 24시간 BGM 리스너"

    # 6. [수정됨] CSV 기반 추천 로직 (카테고리 없이 전체 탐색)
    recommendation = "이미 훌륭한 소비 습관을 가지고 계십니다!"

    if REC_LIST: # CSV가 로드되어 있다면
        my_services = summary['serviceName'].tolist() # 내가 이미 구독중인 목록

        # 1. 내가 구독하지 않은 서비스만 필터링
        candidates = [item for item in REC_LIST if item['service_name'] not in my_services]

        if candidates:
            # 2. 추천 전략: CSV 목록 중 가장 저렴한(가성비) 서비스 추천
            # (카테고리가 없으므로 가격 경쟁력을 우선으로 추천)
            best_candidate = min(candidates, key=lambda x: x.get('cost', 999999))

            svc_name = best_candidate.get('service_name', '알 수 없음')
            svc_benefit = best_candidate.get('benefit_summary', '혜택 정보 없음')
            svc_cost = best_candidate.get('cost', 0)

            recommendation = f"💡 '{svc_name}'({int(svc_cost)}원)은 어떠세요? {svc_benefit}"
        else:
            recommendation = "와우! 저희 DB에 있는 모든 서비스를 이미 알고 계시네요."

    return persona, report_list, recommendation

# ---------------------------------------------------------
# Flask 라우트
# ---------------------------------------------------------
@app.route("/analyze", methods=['POST'])
def analyze():
    try:
        json_data = request.json
        data_list = json_data.get('logs', [])

        if not data_list:
             return jsonify({
                "user_persona": "데이터 부족",
                "inefficiency_report": [],
                "recommendation": "데이터를 먼저 생성해주세요."
            })

        df = pd.DataFrame(data_list)
        persona, report, rec_msg = analyze_comprehensive(df)

        return jsonify({
            "user_persona": persona,
            "inefficiency_report": report,
            "recommendation": rec_msg
        })

    except Exception as e:
        print("Error:", e)
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run()

🚀 [서버 시작] https://hymnodical-superadequately-esmeralda.ngrok-free.dev
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Dec/2025 13:58:44] "POST /analyze HTTP/1.1" 200 -
